<a href="https://colab.research.google.com/github/dtabuena/CrispyMice/blob/main/_Run_All_Nell2_CA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
import shutil
from google.colab import files
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
clear_output(wait=False)



!pip install openpyxl
!pip install XlsxWriter


'''Get Repositories'''
try: shutil.rmtree('/content/EphysLib')
except: None

"run dtabuena's ephys notebooks"
!git clone https://github.com/dtabuena/EphysLib
to_import = [
            'ABF_Quality_Control.ipynb',
            'Basic_Ephys.ipynb',
            'Simple_ABF_tools.ipynb',
            'fun_math.ipynb',
            'importing_abfs_from_dropbox.ipynb',
            'QC_recoding_dataframe.ipynb',
            'Analyzers/input_resistance_analyzer.ipynb',
            'Analyzers/gain_analyzer.ipynb',
            'Analyzers/latencey_analyzer.ipynb',
            'Analyzers/IV_analyzer.ipynb',
            'Analyzers/Vm_analyzer.ipynb',
            'Analyzers/membrane_analyzer.ipynb',
            'Analyzers/rheobase_analyzer.ipynb',
            ]
for i in to_import:
    f = '/content/EphysLib/' + i
    %run $f


try: shutil.rmtree('/content/CrispyMice')
except: None
!git clone https://github.com/dtabuena/CrispyMice
to_import = ['CrispyPipelineFuncs.ipynb']
for i in to_import:
    f = '/content/CrispyMice/' + i
    %run $f


In [ ]:
##### Setup Protocol List
VC_prot = ['VC - MemTest-10ms-160ms',
           'VC - Multi IV - 150ms',]
IC_prot = ['IC - Gain - D20pA',
           'IC - Gain - D50pA',
           'IC - Rheobase',
           'IC - R input',
           'IC - Latentcy 800pA-1s'
           'VC - 3min GapFree'
           'I0 - 3min GapFree']

In [ ]:
dataset = {'data_name': 'NELL2',
           'data_source': "https://www.dropbox.com/sh/kdyo8a4gy4krpp8/AAD-ra8gZJpdsEKq-jk-Yk_da?dl=0", # CA3 Only
           'file_naming_scheme': ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type'],
           }

NELL2_results = crispy_mouse_wrap(dataset,VC_prot,IC_prot,strat_cols=['Rec_date', 'Cell_Type'])

In [ ]:
NELL2_results.keys()
strat_df_dict = NELL2_results['strat_df_dict']


# display( strat_df_dict[list(strat_df_dict.keys())[0]] )


labels = ['Rec_date', 'Virus', 'GenoType', 'Sex', 'Age', 'Slice_Num', 'Cell_num', 'Cell_Type']

possible_cols = list(strat_df_dict[list(strat_df_dict.keys())[0]].columns)
# print(possible_cols)

alt_strat_groups = ['Cm_pc_10.0',
                    'Ra_160.0',
                    'Rm_160.0',
                    'Cmq_160.0',
                    'Gain_(Hz/pA)',
                    'Rheobase',
                    'AP_thresh_US',
                    'fast_after_hyperpol',
                    'Spike_latency_(ms)',
                    'Input_Resistance_MO',
                    'Rmp_mV',
                    [c for c in possible_cols if 'Stim_Levels_(pA)' in c],
                    [c for c in possible_cols if 'Spike_Counts' in c],
                    [c for c in possible_cols if 'IV_Early_(V_stim)' in c],
                    [c for c in possible_cols if 'IV_Steady_State_(I_mean)' in c],
                    [c for c in possible_cols if 'IV_Early_(I_peak)' in c],
                    ]


def add_col( new_df,g,k,v):
        new_col_name = g+'_'+k            
        clean_ser = v[g].reset_index().drop(labels='index',axis=1,inplace=False)
        len_diff =  len(clean_ser)-len(new_df)
        if len_diff>0:
            blank_df = pd.DataFrame( index=range(len_diff),columns=new_df.columns)
            new_df = new_df.append(blank_df,ignore_index=True)
        new_df[new_col_name]=clean_ser
        return new_df

alt_strat_dict = {}
for group in alt_strat_groups:
    new_df = pd.DataFrame()
    for k,v in strat_df_dict.items():
        if isinstance(group, list):
            print(group,'LIST')
            ''' iter the list'''
            group_name = group[0]
            for sub_group in group:
                new_df = add_col( new_df,sub_group,k,v)
        else:
            group_name = group
            new_df = add_col( new_df,group,k,v)
    alt_strat_dict[group_name] = new_df
            





# print(alt_strat_dict.keys())
# display( alt_strat_dict['Cm_pc_10.0'] )
# alt_strat_dict['Cm_pc_10.0'].to_csv('test.csv')
alt_strat_dict

In [ ]:
def write_strat_dfs(strat_dfs, xl_file_name='stratified_data.xlsx'):
    if '.xlsx' not in xl_file_name: xl_file_name = xl_file_name+'.xlsx'
    options = {}
    options['strings_to_formulas'] = False
    options['strings_to_urls'] = False
    writer = pd.ExcelWriter(xl_file_name, options=options)
    for k in strat_dfs.keys():
        cur_df = strat_dfs[k]

        k=k.replace('/','_per_')
        k=k[:31]
        print(str(k))
        cur_df.to_excel(writer, sheet_name=str(k))
    writer.save()
    writer.close()
    files.download(xl_file_name)
    return None

write_strat_dfs(alt_strat_dict, dataset['data_name']+'_results_stratified_alternate')